In [32]:
products = pd.read_csv(
    'https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv',
    sep=';'
)

In [17]:
currencies_to_eur = requests.get(
    'http://data.fixer.io/api/latest?access_key=5227d9e628debf99cabeea740b178492'
).json()['rates']

In [18]:
restcountries_data = requests.get('https://restcountries.eu/rest/v2/all').json()

In [19]:
countrycode_to_currency = {
    country['alpha2Code']: country['currencies'][0]['code']
    for country in restcountries_data
}

countrycode_to_currency = {}
for country in restcountries_data:
    country_code = country['alpha2Code']
    currency = country['currencies'][0]['code']
    countrycode_to_currency[country_code] = currency

In [20]:
def ip_to_countrycode(ip):
    url = f'http://api.ipstack.com/{ip}'
    IPSTACK_KEY = 'd305c0652f17a34f0bfdf555f5b42cfe'
    data = requests.get(url, params={'access_key': IPSTACK_KEY}).json()
    return data['country_code']

In [21]:
ip_ok = products.ip_address.str.contains(
    '^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$'
)

codes = [ip_to_countrycode(ip) for ip in products[ip_ok].ip_address]
products.loc[ip_ok, 'country_code'] = codes

KeyboardInterrupt: 

In [ ]:
splitted = products['price'].str.split(' ')
price = splitted.str[0].astype(float)
currency = splitted.str[1]

products['price'] = price
products['currency'] = currency

products['currency_guessed'] = products['country_code'].replace(countrycode_to_currency)
products['currency'] = products['currency'].combine_first(products['currency_guessed'])

# Keep only products where we have a currency:
products = products[products.currency.notna()].reset_index()

In [16]:
products['price_eur'] = products.price * products.currency.map(currencies_to_eur)

## Ingredients

In [40]:
products.head().infos

0                  May contain sugar
1           Contains peanut and fish
2      Ingredients: mustard and fish
3                    Contains gluten
4    May contain sugar, egg and fish
Name: infos, dtype: object

In [18]:
ing = products.infos\
              .str.replace('^May contain ', 'Contains ')\
              .str.split(' ', 1).str[1]\
              .str.replace(' and ', ', ')\
              .str.get_dummies(sep=', ')

In [19]:
result = pd.concat([products, ing], axis=1)

In [27]:
matrix = products.infos.str.replace('^May contain ', 'Contains ')\
.str.split(' ', 1).str[1].str.replace(' and ', ', ').str.get_dummies(sep=', ')

In [30]:
for col in matrix.columns:
    products[col] = matrix[col]

In [36]:
matrixh = matrix.head(198)

In [39]:
pd.concat([products, matrixh], axis=1)

,username,ip_address,product,price,infos,egg,fish,gluten,milk,mustard,peanut,soja,sugar
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,ccannop5f,174.5.73.129,Nantucket Orange Juice,713.6,"Contains gluten, mustard, sugar, milk and fish",0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
196,lprovis5g,191.69.45.257,"Jam - Strawberry, 20 Ml Jar",107.78,Contains gluten and sugar,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",447.17 RUB,May contain gluten and soja,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
198,mjablonski5i,44.133.211.182,"Mushroom - Chantrelle, Fresh",632.29,Ingredients: soja and sugar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
